# Student Bayes Network

In [302]:
#load libraries and set plot parameters

## Building dependences by graph
1. We set the states of the variables.
    - Variables that not have children have a probability asociated to each state.
    - Variables that have children have a dictionary with the children state asociated to a probability.
2. We set the variables that have children.
3. Variables:
    - Difficulty
    - Intelligence
    - SAT
    - Grade
    - Letter

In [303]:
def build_graph():
    D = {'states': {0: 0.6, 1: 0.4},
         'children': None}
    
    I = {'states': {0: 0.7, 1: 0.3},
         'children': None}
    
    G = {'states':{ 1: {(0,0):0.3, (0,1):0.05, (1,0):0.9 , (1,1):0.5},
                                 2: {(0,0):0.4, (0,1):0.25, (1,0):0.08, (1,1):0.3},
                                 3: {(0,0):0.3, (0,1):0.7 , (1,0):0.02, (1,1):0.2}},
         'children': {'Intelligence':0, 'Difficulty':1}}
    
    S = {'states': {0: {0:0.95, 1:0.2},
                     1: {0:0.05, 1:0.8}},
         'children': {'Intelligence':0}}
    
    L = {'states':{0: {1:0.1 , 2:0.4, 3:0.99},
                     1: {1:0.9 , 2:0.6, 3:0.01}},
         'children': {'Grade':0}}

    graph = {'Difficulty': D, 'Intelligence': I, 'Grade': G, 'SAT': S, 'Letter': L}
    
    return graph

We need a function to truncate probabilities values in order to get a better vizualization of probabilities.

In [304]:
def truncateDecimal(f):
    f = f*1000/1000
    s = "%.4f" % f
    return float(s)

## Getting probabilities on each variable

1. We verify if a variables have children.
    - If a variable have children we calculate calculate the probability first of his children.
    - If a variable don't have children we normalize directly the variable's probability.
    
2. Normalize:
    - Go through each state of the variables.
    - Get state's probability.
    - Multiply by the probability of the son's state.
    - Divide each probability by the sum of all of them.

In [338]:
def getProb(reset_bn=True, evidences=None, verbose=True):
    if reset_bn:
        BayNet = build_graph()
    
    def normalize(states, children, BayNet,deep):
        sump = 0.0
        res_prob  = {}
        if children:
            rev_chdic = dict(zip(children.values(), children.keys()))
        for key_state, children_states in states.items():
            if isinstance(children_states,(int,float)):
                res_prob[key_state] = children_states
                sump               += children_states
            else:
                sum_state = 0
                for children_state,p in children_states.items():
                    prob_state = p
                    if isinstance(children_state, (int,float)):
                        if rev_chdic[0] in res:
                            prob_state *= res[rev_chdic[0]][children_state]

                    else:
                        for i, child_state in enumerate(children_state):
                            if rev_chdic[i] in res:
                                prob_state *= res[rev_chdic[i]][child_state]
                            else:
                                prob_state *=  BayNet[rev_chdic[i]]['states'][child_state]
                    sum_state      += prob_state
                res_prob[key_state] = sum_state
                sump               += sum_state
            
        for key, val in res_prob.items():
            res_prob[key] = truncateDecimal(val/sump)
        return res_prob
    
    def setProb(res, key, BayNet, deep):
        if BN[key]['children']:
            deep +=1
            for child in BayNet[key]['children']:
                if verbose:
                    print('\t'*deep + 'child',child, '<-',BayNet[key]['children'] )
                setProb(res, child, BayNet,deep) 
            if verbose:
                print('\t'*(deep+1) + 'normalize ->',key)
            res[key] = normalize(BayNet[key]['states'],BayNet[key]['children'], BayNet,0)
            #print('**',res)
        else:
            if key not in res:
                if verbose:
                    print('\t'*(deep+1) + 'normalize ->',key)
                res[key] = normalize(BayNet[key]['states'], None, BayNet,deep)
                #print('**',res)
    
    def setEvidence(ev, res):
        ev_prob = {}
        for ev_state in BayNet[ev[0]]['states']:
            if ev_state == ev[1]:
                ev_prob[ev_state] = 1.0
            else:
                ev_prob[ev_state] = 0.0
        res[ev[0]] = ev_prob
    
    def setEviChildren(res, children, ev_parent, BayNet):
        if children:
            for child in children:
                res[child] = normalize(BayNet[child]['states'], BayNet[child]['children'], BayNet, 0)
                for s, p in res[child].items():
                    res[child][s] = ev_parent[s]*res[child][s]
                
                res[child] = normalize(res[child], None, BayNet, 0)
                setEviChildren(res, BayNet[child]['children'], res[child], BayNet)
        
    res = {}

    if evidences:
        for evidence in evidences:
            if evidence[0] in BayNet:
                setEvidence(evidence, res)
                setEviChildren(res, BayNet[evidence[0]]['children'], BayNet[evidence[0]]['states'][evidence[1]], BayNet)                        
            else:
                print("Evidence's name not found!\n")

    for key, node in BN.items():
        if verbose:
            print('key :', key)
        if key not in res:
            setProb(res, key, BayNet,0) 
    return res

In [326]:
def printProb(PD):
    for key, val in PD.items():
        print('{:10}\t\t: {}'.format(key, val)) 

How works **getProb**:

In [327]:
PD = getProb(reset_bn=True)

key : Intelligence
	normalize -> Intelligence
key : Letter
	child Grade <- {'Grade': 0}
		child Intelligence <- {'Intelligence': 0, 'Difficulty': 1}
		child Difficulty <- {'Intelligence': 0, 'Difficulty': 1}
			normalize -> Difficulty
			normalize -> Grade
		normalize -> Letter
key : Difficulty
key : SAT
	child Intelligence <- {'Intelligence': 0}
		normalize -> SAT
key : Grade


### Probabilities

In [328]:
printProb(PD)

Intelligence		: {0: 0.7, 1: 0.3}
Letter    		: {0: 0.4977, 1: 0.5023}
Difficulty		: {0: 0.6, 1: 0.4}
SAT       		: {0: 0.725, 1: 0.275}
Grade     		: {1: 0.362, 2: 0.2884, 3: 0.3496}


## Comparation with SAMIAM

We have the same probability distribution as Samiam

In [329]:
%%html
<img src='imgs/disprobinit.png'>

Function to get the probability distribution of one variables.

In [330]:
def query(key, prob_dist):
    return prob_dist[key]   

In [331]:
query("Intelligence", PD)

{0: 0.7, 1: 0.3}

In [332]:
query("Letter", PD)

{0: 0.4977, 1: 0.5023}

## Setting an evidence

In [339]:
PD = getProb(reset_bn=True, evidences = [('Intelligence',0)], verbose=False)

In [340]:
printProb(PD)

Intelligence		: {0: 1.0, 1: 0.0}
Letter    		: {0: 0.6114, 1: 0.3886}
Difficulty		: {0: 0.6, 1: 0.4}
SAT       		: {0: 0.95, 1: 0.05}
Grade     		: {1: 0.2, 2: 0.34, 3: 0.46}


We have the same probability distribution as Samiam also setting an evidence.

In [322]:
%%html
<img src='imgs/evi1.png'>

In [341]:
PD = getProb(reset_bn=True, evidences = [('Difficulty',1)], verbose=False)

In [342]:
printProb(PD)

Intelligence		: {0: 0.7, 1: 0.3}
Letter    		: {0: 0.669, 1: 0.331}
Difficulty		: {0: 0.0, 1: 1.0}
SAT       		: {0: 0.725, 1: 0.275}
Grade     		: {1: 0.185, 2: 0.265, 3: 0.55}


In [343]:
%%html
<img src='imgs/evi2.png'>

## Setting two evidences

In [344]:
evidences = [('Intelligence',0),('Difficulty',1)]
PD = getProb(reset_bn=True, evidences = evidences, verbose=False)

In [345]:
printProb(PD)

Intelligence		: {0: 1.0, 1: 0.0}
Letter    		: {0: 0.798, 1: 0.202}
Difficulty		: {0: 0.0, 1: 1.0}
SAT       		: {0: 0.95, 1: 0.05}
Grade     		: {1: 0.05, 2: 0.25, 3: 0.7}


In [346]:
%%html
<img src='imgs/evi3.png'>

## Setting evidence on *SAT* variable

In [347]:
evidences = [('SAT',0)]
PD = getProb(reset_bn=True, evidences = evidences, verbose=False)

In [348]:
printProb(PD)

Intelligence		: {0: 0.9172, 1: 0.0828}
Letter    		: {0: 0.58, 1: 0.42}
Difficulty		: {0: 0.6, 1: 0.4}
SAT       		: {0: 1.0, 1: 0.0}
Grade     		: {1: 0.2447, 2: 0.3258, 3: 0.4295}


In [349]:
%%html
<img src='imgs/evi4.png'>

In [350]:
evidences = [('Grade',3)]
PD = getProb(reset_bn=True, evidences = evidences, verbose=False)

KeyError: 0